In [1]:
import utils
import models.categorical
import math
import pickle
import numpy as np
import pandas as pd
from datetime import date
import os.path
from os import path

from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

## Settings

In [2]:
start_date = date(2010, 1, 1)
end_date = date(2021, 1, 1)
futuresList = utils.futuresList
FUTURE_INDUSTRY_PATH = "utils/future_industry_mapping.csv"
FUTURE_INDUSTRY = pd.read_csv(FUTURE_INDUSTRY_PATH)
FUTURE_INDUSTRY = FUTURE_INDUSTRY.set_index("Ticker")

In [3]:
fourCandleHammerParams = [
    {
        "N": [30, 50],
        "highFactor": [0.95, 0.85],
        "lowFactor": [1.05, 1.15],
        "macro_analysis": [True, False]
    }
]

emaParams = [
    {
        "shortTermDays": [10, 20],
        "longTermDays": [30, 40],
        "NDays": [50, 60],
        "macro_analysis": [True, False]
    }
]

swingSetupParams = [
    {
        "shortTermDays": [20, 30],
        "longTermDays": [40, 60],
        "NDays": [80, 100],
        "macro_analysis": [True, False]
    }
]

## Hyperparameter Tuning for FourCandleHammer Strategy

In [4]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(fourCandleHammerParams))
y_var = "LONG_SHORT"
file_dir = "fourCandleHammer/perc/"

for future in tqdm(futuresList):
    if path.exists(f"model_metrics/categorical/{file_dir}{future}.csv"): continue
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    X_vars.extend(utils.generate_macroeconomic_indicators_techIndicators(future, macro_indicators=True))
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        df_with_strategy = utils.fourCandleHammer(X_df, param_set['N'], param_set['highFactor'], param_set['lowFactor'], future, param_set['macro_analysis'])
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            df_strategy = df_with_strategy, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]
        
    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

100%|██████████| 88/88 [48:08<00:00, 32.83s/it]


## Save Meta Predictions

In [4]:
file_dir = "fourCandleHammer/perc/"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    X_vars.extend(utils.generate_macroeconomic_indicators_techIndicators(future, macro_indicators=True))
    
    models.categorical.save_meta_predictions_techIndicators(
        path = file_dir, metric="accuracy_SMA", future = future,
        X_vars = X_vars, y_var = y_var, model_name = "fourCandleHammer"
    )

100%|██████████| 88/88 [10:55<00:00,  7.45s/it]


## Save FourCandleHammer Model

In [5]:
for future in futuresList:
    # prepare X variables
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    X_vars.extend(utils.generate_macroeconomic_indicators_techIndicators(future, macro_indicators=True))
    
    # save model
    # metric="opp_cost_SMA"
    models.categorical.save_model_techIndicators(path='fourCandleHammer/perc', metric="accuracy_SMA", 
                                                model_wrapper=models.categorical.fourCandleHammerWrapper, 
                                                future=future, X_vars=X_vars, y_var="LONG_SHORT")


## Hyperparameter Tuning for EMA Strategy

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(emaParams))
y_var = "LONG_SHORT"
file_dir = "ema/perc/"

for future in tqdm(futuresList):
    if path.exists(f"model_metrics/categorical/{file_dir}{future}.csv"): continue
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    X_vars.extend(utils.generate_macroeconomic_indicators_techIndicators(future, macro_indicators=True))
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        if param_set['shortTermDays'] > param_set['longTermDays']: continue
        df_with_strategy = utils.ema_strategy(X_df, param_set['shortTermDays'], param_set['longTermDays'], param_set['NDays'], future, param_set['macro_analysis'])
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            df_strategy = df_with_strategy, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

## Save Meta Predictions

In [ ]:
file_dir = "ema/perc/"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    X_vars.extend(utils.generate_macroeconomic_indicators_techIndicators(future, macro_indicators=True))
    
    models.categorical.save_meta_predictions_techIndicators(
        path = file_dir, metric="accuracy_SMA", future = future,
        X_vars = X_vars, y_var = y_var, model_name = "ema_strategy"
    )

## Save EMA Model

In [ ]:
for future in futuresList:
    # prepare X variables
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # save model
    # metric="opp_cost_SMA"
    models.categorical.save_model_techIndicators(path='ema/perc', metric="accuracy_SMA", 
                                                model_wrapper=models.categorical.emaStrategyWrapper, 
                                                future=future, X_vars=X_vars, y_var="LONG_SHORT")

## Hyperparameter Tuning for Swing Setup Strategy

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(swingSetupParams))
y_var = "LONG_SHORT"
file_dir = "swingSetup/perc/"

for future in tqdm(futuresList):
    if path.exists(f"model_metrics/categorical/{file_dir}{future}.csv"): continue
    print(future)

    # load data
    df = utils.prepare_data(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    X_vars.extend(utils.generate_macroeconomic_indicators_techIndicators(future, macro_indicators=True))
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]

    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        if param_set['shortTermDays'] > param_set['longTermDays']: continue
        df_with_strategy = utils.swing_setup(X_df, param_set['shortTermDays'], param_set['longTermDays'], param_set['NDays'], future, param_set['macro_analysis'])
        # print("Gotten strategy, now walk fwd")
        win_results, agg_results = models.categorical.walk_forward_techIndicators(
            df_strategy = df_with_strategy, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        # print("Done walk fwd")
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

## Save Meta Predictions

In [ ]:
file_dir = "swingSetup/perc/"
y_var = "LONG_SHORT"

for future in tqdm(utils.futuresList):
    print(future)
    
    # generate X vars
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    X_vars.extend(utils.generate_macroeconomic_indicators_techIndicators(future, macro_indicators=True))
    
    models.categorical.save_meta_predictions_techIndicators(
        path = file_dir, metric="accuracy_SMA", future = future,
        X_vars = X_vars, y_var = y_var, model_name = "swing_setup"
    )

## Save Swing Setup Model

In [ ]:
for future in futuresList:
    # prepare X variables
    X_vars = ['OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
    
    # save model
    # metric="opp_cost_SMA"
    models.categorical.save_model_techIndicators(path='swingSetup/perc', metric="accuracy_SMA", 
                                                model_wrapper=models.categorical.swingSetupWrapper, 
                                                future=future, X_vars=X_vars, y_var="LONG_SHORT")